In [ ]:
# %% [markdown]
# # 05 - Technical Analysis with TA-Lib
# ## Quantitative Analysis of Stock Prices
#
# **Techniques:** Moving Averages, RSI, MACD, Bollinger Bands, Stochastic

In [ ]:


import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sys.path.append('../src')
from src.technical_analyzer import TechnicalAnalyzer
from src.visualization import FinancialVisualizer
from src.config import TICKERS

print("✅ Technical analysis packages installed!")

In [ ]:
# %%
# Initialize analyzers
print("🔧 INITIALIZING TECHNICAL ANALYSIS...")
tech_analyzer = TechnicalAnalyzer()
visualizer = FinancialVisualizer()

In [ ]:
# %%
# Load and prepare stock price data
print("📥 LOADING STOCK PRICE DATA...")

# Try to load existing price data or download new data
price_data = {}
all_metrics = {}

for ticker in TICKERS:
    filepath = f'../data/prices/{ticker}_prices.csv'
    if os.path.exists(filepath):
        print(f"✅ Loading {ticker} price data from file...")
        df = tech_analyzer.load_price_data(filepath)
        if not df.empty:
            price_data[ticker] = df
    else:
        print(f"⚠️  {ticker} price file not found. Please ensure price data is in data/prices/")
        # Create sample data for demonstration
        print(f"🔧 Creating sample data for {ticker}...")
        dates = pd.date_range('2023-01-01', '2024-01-01', freq='D')
        sample_data = pd.DataFrame({
            'Date': dates,
            'Open': np.random.normal(150, 20, len(dates)).cumsum() + 100,
            'High': np.random.normal(155, 25, len(dates)).cumsum() + 100,
            'Low': np.random.normal(145, 25, len(dates)).cumsum() + 100,
            'Close': np.random.normal(150, 20, len(dates)).cumsum() + 100,
            'Volume': np.random.randint(1000000, 50000000, len(dates)),
            'Stock': ticker
        })
        sample_data['High'] = sample_data[['Open', 'Close', 'High']].max(axis=1) + np.random.uniform(1, 5)
        sample_data['Low'] = sample_data[['Open', 'Close', 'Low']].min(axis=1) - np.random.uniform(1, 5)
        
        os.makedirs('../data/prices', exist_ok=True)
        sample_data.to_csv(filepath, index=False)
        price_data[ticker] = tech_analyzer.load_price_data(filepath)

print(f"✅ Loaded price data for {len(price_data)} companies")

In [ ]:
# %%
# Perform technical analysis for each stock
print("🎯 PERFORMING TECHNICAL ANALYSIS...")
technical_results = {}

for ticker, df in price_data.items():
    print(f"📊 Analyzing {ticker}...")
    analyzed_df = tech_analyzer.analyze_stock(f'../data/prices/{ticker}_prices.csv')
    if not analyzed_df.empty:
        technical_results[ticker] = analyzed_df
        # Calculate performance metrics
        metrics = tech_analyzer.calculate_performance_metrics(analyzed_df)
        all_metrics[ticker] = metrics
        print(f"   • Return: {metrics.get('total_return', 0):.2f}%")
        print(f"   • Volatility: {metrics.get('volatility', 0):.2f}%")
        print(f"   • Sharpe: {metrics.get('sharpe_ratio', 0):.2f}")

print("✅ Technical analysis completed!")

In [ ]:
# %%
# Display technical indicators summary
print("📈 TECHNICAL INDICATORS SUMMARY")
print("=" * 70)

for ticker, df in technical_results.items():
    if not df.empty:
        latest = df.iloc[-1]
        print(f"\n🏢 {ticker}:")
        print(f"   • Price: ${latest['Close']:.2f}")
        if 'RSI' in df.columns:
            rsi_status = "OVERSOLD" if latest['RSI'] < 30 else "OVERBOUGHT" if latest['RSI'] > 70 else "NEUTRAL"
            print(f"   • RSI: {latest['RSI']:.1f} ({rsi_status})")
        if 'MACD' in df.columns:
            macd_signal = "BULLISH" if latest['MACD'] > latest['MACD_Signal'] else "BEARISH"
            print(f"   • MACD: {latest['MACD']:.3f} ({macd_signal})")
        if 'SMA_20' in df.columns and 'SMA_50' in df.columns:
            ma_signal = "BULLISH" if latest['SMA_20'] > latest['SMA_50'] else "BEARISH"
            print(f"   • MA Signal: {ma_signal} (20-day: {latest['SMA_20']:.2f}, 50-day: {latest['SMA_50']:.2f})")

In [ ]:
# %%
# Save technical analysis results
print("\n💾 SAVING TECHNICAL ANALYSIS RESULTS...")
os.makedirs('../data/processed', exist_ok=True)

for ticker, df in technical_results.items():
    if not df.empty:
        df.to_csv(f'../data/processed/{ticker}_technical_analysis.csv')
        print(f"✅ Saved {ticker} technical analysis")

# Save performance metrics
metrics_df = pd.DataFrame(all_metrics).T
metrics_df.to_csv('../data/processed/technical_performance_metrics.csv')
print("✅ Saved performance metrics")

print("\n🎉 TECHNICAL ANALYSIS COMPLETE!")
print("➡️ Proceed to 06_price_visualization.ipynb")